In [ ]:
# login
import sys
import os

project_root = os.path.dirname(os.path.dirname(os.getcwd()))
if project_root not in sys.path:
    sys.path.append(project_root)

from utils.brainLogin import get_session

session = get_session()

response = session.post("https://api.worldquantbrain.com/authentication")
print(response.status_code)
print(response.json())

In [ ]:
# get datafields
from utils.brainGetDataFields import get_datafields
from utils.brainDatafieldsSearchScopeConfig import get_search_scope 

search_scope = get_search_scope()
datafields = get_datafields(session, search_scope, dataset_id='fundamental6', search='', field_type='MATRIX')
datafields_list_fnd6 = datafields['id'].values
print(datafields_list_fnd6)



In [3]:
# template <group_compare_op>(<ts_compare_op>(<company_fundamentals>, <days>), <group>)

In [ ]:
# 将datafield和operator替换到Alpha模板（框架）中 => group_rank(ts_rank({fundamental model data}, 252), industry), 批量生成Alpha
# 定义操作符列表
group_compare_op = ['group_rank', 'group_zscore', 'group_neutralize']
ts_compare_op = ['ts_rank', 'ts_zscore', 'ts_av_diff'] # 之前获取的fundamental6数据字段列表
company_fundamentals = datafields_list_fnd6 
days = [600, 200]
group = ['market', 'industry', 'subindustry', 'sector', 'densify(pv13_h_f1_sector)'] # 分组方式

# 初始化alpha表达式列表
alpha_expressions = []

# 嵌套循环生成所有可能的组合
for gco in group_compare_op:
    for tco in ts_compare_op:
        for cf in company_fundamentals:
            for d in days:
                for grp in group:
                    # 生成alpha表达式并添加到列表
                    alpha_expressions.append(f"{gco}({tco}({cf}, {d}), {grp})")

# 打印生成的表达式数量
print(f'there are total {len(alpha_expressions)} alpha expressions')

In [5]:
# 生成alpha列表
from utils.brainSimulationConfig import get_simulation_data

alpha_list = []
for alpha_expression in alpha_expressions:
    simulation_data = get_simulation_data(alpha_expression)
    alpha_list.append(simulation_data)

In [ ]:
print(alpha_list.__len__())
print(alpha_list[0])

In [ ]:
from utils.brainSimulation import run_alpha_simulation, rerun_alphas

rerun_alphas("pending")


In [ ]:
from utils.brainSimulationRecord import check_progress

check_progress('20250113_181626')